In [16]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import math
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

In [17]:
def lapsm(n_label_items, vocab, word_counts, word, text_label):
    a = word_counts[text_label][word] + 1
    b = n_label_items[text_label] + len(vocab)
    return math.log(a/b)

In [18]:
def label_group(x, y, labels):
    data = {}
    for l in labels:
        data[l] = x[np.where(y == l)]
    return data

In [19]:
def fit(x, y, labels):
    n_label_items = {}
    log_label_priors = {}
    n = len(x)
    grouped_data = label_group(x, y, labels)
    for l, data in grouped_data.items():
        n_label_items[l] = len(data)
        log_label_priors[l] = math.log(n_label_items[l] / n)
    return n_label_items, log_label_priors

In [20]:
def predict(n_label_items, vocab, word_counts, log_label_priors, labels, x):
    result = []
    for text in x:
        label_scores = {l: log_label_priors[l] for l in labels}
        words = set(w_tokenizer.tokenize(text))
        for word in words:
            if word not in vocab: continue
            for l in labels:
                log_w_given_l = lapsm(n_label_items, vocab, word_counts, word, l)
                label_scores[l] += log_w_given_l
        result.append(max(label_scores, key=label_scores.get))
    return result

In [21]:
def get_word_counts(X, train_labels, vocab):
    word_counts = {"sport": {}, "business": {}, "politics": {}, "entertainment": {}, "tech": {}}
    for i in range(X.shape[0]):
        l = train_labels[i]
        for j in range(len(vocab)):
            if(vocab[j] in word_counts[l].keys()):
                word_counts[l][vocab[j]] += X[i][j]
            else:
                word_counts[l][vocab[j]] = X[i][j]
    return word_counts

In [22]:
def accuracy_score(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return (correct / float(len(actual))) * 100.0

## Problem Overview

In this assignment, we will try to determine which category a news article belongs to
among five categories (Sport, Business, Politics, Entertainment, Tech).
We will implement a Naive Bayes classifier and verify it’s performance on English News
Dataset. As we learned in class, Naive Bayes is a simple classification algorithm
that makes an assumption about the conditional independence of features, but it works
quite well in practice.

# Part 1

    Here is the number of articles for each categories:
    Sports: 346
    Business: 336
    Politics: 274
    Entertainment: 273
    Tech: 261

    Here is the three words that we choose with their statistics (frequencies):
    Sports:
        the: 253/346
        to: 103/346
        in: 58/346
    Business:
        the: 255/336
        to: 101/336
        in: 66/336
    Politics:
        the: 220/274
        to: 138/274
        of: 69/274
    Entertainment:
        the: 202/273
        to: 45/273
        and: 39/273
    Tech:
        the: 198/261
        to: 127/261
        of: 93/261

    Is that feasible?

    Of course not. Although these words have the highest frequencies, we cannot determine the category of the article based on just the freqıency value. We should have other parameters to do this task. In the following parts of this assignment we will yse TF-IDF for this task.

# Part 2

    In this part, we will use bag of words (BoW) data structure to vectorize our words in the articles and then, apply Naive Bayes to them. We will use both unigram and bigram for this task.

In [23]:
# read the csv file
df = pd.read_csv("English Dataset.csv", encoding='cp1254')
data = df.to_numpy()  # convert it to numpy array

In [24]:
# shuffle the data
np.random.shuffle(data)

# train-test split
train_sentences, test_sentences, train_labels, test_labels = train_test_split(data[:,1], data[:,2], test_size=0.2)

In [25]:
# PART 2
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
print("PART 2:")
# apply naive bayes to unigram
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(train_sentences)
vocab = vectorizer.get_feature_names_out()
X = X.toarray()
word_counts = get_word_counts(X, train_labels, vocab)

labels = ["sport", "business", "politics", "entertainment", "tech"]
n_label_items, log_label_priors = fit(train_sentences,train_labels,labels)
pred = predict(n_label_items, vocab, word_counts, log_label_priors, labels, test_sentences)
print("Accuracy of unigram: ", accuracy_score(test_labels,pred))

PART 2:
Accuracy of unigram:  88.9261744966443


In [26]:
# apply naive bayes to bigram
vectorizer2 = CountVectorizer(ngram_range=(2,2), analyzer='word')
X2 = vectorizer2.fit_transform(train_sentences)
vocab2 = vectorizer2.get_feature_names_out()
X2 = X2.toarray()
word_counts2 = get_word_counts(X2, train_labels, vocab2)

pred = predict(n_label_items, vocab2, word_counts2, log_label_priors, labels, test_sentences)
print("Accuracy of bigram: ", accuracy_score(test_labels,pred))

Accuracy of bigram:  22.818791946308725


As you can see from the accuracies abnove, the unigram gave us a better result (approx. 90%). In this implementation, the bigram gave us a bad result (around 25%) and takes too much time to execute (nearly 10 mins). When we run our algorithm, the countVectorizer takes all of the words in the article and forms a dictionary to return their frequencies, then applies naive bayes on them. We have nearly 20000 unique words. On the other hand, the bigram has nearly 220000 words. When the bigram forms its dictionary, it takes every combination of two sequential words, including the stop words. This is the reason why the bigram takes too long and gives poor results.

# Part 3

In this part, we are going to list 10 words whose presence strongly predicts that an article belongs to specific category for each five categories and another 10 words whose absence strongly predicts the same categories. Later on, we will compare the performance of using or not using the stop words.

In [27]:
# PART 3
# a
category = {"sport": [], "business": [], "politics": [], "entertainment": [], "tech": []}
for i in range(len(train_labels)):
    category[train_labels[i]].append(train_sentences[i])

train_narrowed = []
labels_narrowed = []
for cat in category.keys():
    tfidfvectorizer = TfidfVectorizer(analyzer='word')
    tfidf_wm = tfidfvectorizer.fit_transform(category[cat])
    tfidf_tokens = tfidfvectorizer.get_feature_names_out()

    matrix = tfidf_wm.toarray()
    all_texts = []
    all_words = []
    for i in matrix:
        max_ind = np.argpartition(i, -20)[-20:]
        all_texts.extend(i[max_ind])
        all_words.extend(tfidf_tokens[max_ind])

    ind = np.argpartition(all_texts, -20)[-20:]
    best10 = {}
    for i in range(len(np.array(all_words)[ind])):
        if(np.array(all_words)[ind][i] not in best10.keys()):
            best10[np.array(all_words)[ind][i]] = np.array(all_texts)[ind][i]
    best10 = {k: v for k, v in sorted(best10.items(), reverse=True ,key=lambda item: item[1])}
    best10 = {k: best10[k] for k in list(best10)[:10]}

    df = pd.DataFrame.from_dict(best10, orient="index")
    print(df)
    
    min_texts = []
    min_words= []

    mat = np.copy(matrix)
    mat[mat == 0] = 999
    for i in mat:
        min_ind = np.argpartition(i, 20)[:20]
        min_texts.extend(i[min_ind])
        min_words.extend(tfidf_tokens[min_ind])

    ind_min = np.argpartition(min_texts, 20)[:20]
    bottom10 = {}
    for i in range(len(np.array(min_words)[ind_min])):
        if(np.array(min_words)[ind_min][i] not in bottom10.keys()):
            bottom10[np.array(min_words)[ind_min][i]] = np.array(min_texts)[ind_min][i]
    bottom10 = {k: v for k, v in sorted(bottom10.items(), key=lambda item: item[1])}
    bottom10 = {k: bottom10[k] for k in list(bottom10)[:10]}
    dfmin = pd.DataFrame.from_dict(bottom10, orient="index")
    dfmin.head(10)

    train_narrowed.extend(all_words)
    temp= (cat+" ")*len(all_words)
    t_list = temp.split(" ")
    labels_narrowed.extend(t_list[:-1])

                  0
pountney   0.618079
soderling  0.596906
glazer     0.593229
mido       0.588578
students   0.566474
ivanovic   0.565251
mirza      0.561022
conte      0.557994
harriers   0.551360
solskjaer  0.549857
                0
fiat     0.707974
nestle   0.687244
absa     0.672968
ssl      0.651372
qantas   0.645823
mci      0.645604
metlife  0.641346
turkey   0.640364
gm       0.635750
women    0.635571
                  0
fraud      0.642757
hague      0.605789
turkey     0.582320
the        0.570803
hunting    0.553382
blackpool  0.544165
sainsbury  0.532860
roma       0.531554
arrested   0.517370
casinos    0.514653
                  0
godzilla   0.702534
edwards    0.623879
christmas  0.621976
sky        0.605766
aguilera   0.583050
ice        0.580039
clark      0.571119
duran      0.569054
wal        0.562631
elvis      0.552088
                  0
commodore  0.692548
p2p        0.688422
uwb        0.608684
bt         0.599916
cabir      0.572749
library    0.565201
sk

Each pair of the dataframes above represent the best and the worst TF-IDF values for each category.
To give an example, the first dataframe is the best TF-IDF results for the category "sport", and the second one is the worst values for the same category.
These best10 values are the strongest words for identifying the category of the article. Same goes for the worst 10 words as well. Rather than their presence, their "absence" are crucial for identifying the article correctly. 

In this assignment, we have use the strongest words for the naive bayes implementation. We have narrowed down our train data by choosing the strongest words, and then implemented our algorithm. Compering to the unigram implementation of part 2, our vocabulary has half less words. Thus, our execution time is much shorter. In case of accuracy score, this choice of words gives slightly less accuracy then choosing all words (about 6%) but considering the time efficiency, this is a reasonable trade-off.


If we remove stop words from our vocabulary for this implementation, you will see that in both time efficiency and accuracy score, this implementation will be better.

In [28]:
print("PART 3:")
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(train_narrowed)
vocab = vectorizer.get_feature_names_out()
X = X.toarray()
word_counts = get_word_counts(X, labels_narrowed, vocab)

n_label_items, log_label_priors = fit(train_sentences,train_labels,labels)
pred = predict(n_label_items, vocab, word_counts, log_label_priors, labels, test_sentences)
print("Accuracy of best10: ", accuracy_score(test_labels,pred))

PART 3:
Accuracy of best10:  86.91275167785236


In [29]:
    #b
    vectorizer = CountVectorizer(stop_words=ENGLISH_STOP_WORDS)
    X = vectorizer.fit_transform(train_narrowed)
    vocab = vectorizer.get_feature_names_out()
    X = X.toarray()
    word_counts = get_word_counts(X, labels_narrowed, vocab)

    n_label_items, log_label_priors = fit(train_sentences,train_labels,labels)
    pred = predict(n_label_items, vocab, word_counts, log_label_priors, labels, test_sentences)
    print("Accuracy of non-stop_word best10: ", accuracy_score(test_labels,pred))

Accuracy of non-stop_word best10:  94.29530201342283


When we are interpreting our model it makes sense to remove stop words from our vectorizer because these words are generally present in almost any news articles in our database and as such they do not help in determining which category an article belongs to. It might make sense to keep stop words when we are using Bag of Word(BoW) with n-grams where n>1. For example, for category "Entertainment" with trigram model, we might have adjacent words such "Whack a Mole". When these words are taken one by one they might not help much in determining our category but when they are together like this, these words gain a meaning that helps us much more in determining its category. So we should decide on whether removing or keeping stop words from our data based on how we are interpreting the data, because its results will change depending on which methods we are going to use.